# NLP 
- analiza jezykow uzywanych przez ludzi

Definujemy modele ktore moga zrozumiec dany jezyk, sklasyfikowac tresc, wygenerowac utowry. 


Anagram slowa romantyk to markotny. W Kodzie ASCII reprezentuje te slowa te same liczby. Co utrudnia zdudowanie modelu rozumienia tekstu.\
Rozwiazeniem tego problemy jest zakodowanie calych slow zamiast zawartych w nich liter. Romatyk to liczba x, markotyny to y.


Tokenizacja: 'Bardzo lubie jesc makaron' mozemy zakodowac za pomoca [1,2,3,4], natmoiast zdanie 'Bardzo lubie jesc pizze' mozemy zakodowac [1,2,3,5], mozemy stwierdzic ze zdania maja podobne znaczenie ze wzgledu ze ciagi sa podobne.

In [1]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
sentences = ['Today is a sunny day',
            'Today is a ruiny day',
            'Is today a sunny day?']

tokenizer = Tokenizer(num_words=100)  
tokenizer.fit_on_texts(sentences) # tokazynacja zdan 
print(tokenizer.word_index) 

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'ruiny': 6}


# Zamiana zdan na sekwencje 

In [5]:
sequences = tokenizer.texts_to_sequences(sentences)
print(sequences) 

[[1, 2, 3, 5, 4], [1, 2, 3, 6, 4], [2, 1, 3, 5, 4]]


# Token typu OOV out of vocabulary 

In [6]:
test_data = ['Today is a snowing day',
            'Will it be a rainy day tommorow?']

test_sequences = tokenizer.texts_to_sequences(test_data)
print(tokenizer.word_index)
print(test_sequences) 

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'ruiny': 6}
[[1, 2, 3, 4], [3, 4]]


W powyzszym przykladznie widac ze stracilismy kontekst i znaczenie zdan. 
W takim przypadku moze pomoc wykorzystanie tokenu 'oov" 

In [9]:
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)

test_sequences = tokenizer.texts_to_sequences(test_data)
print(tokenizer.word_index)
print(test_sequences)



{'<OOV>': 1, 'today': 2, 'is': 3, 'a': 4, 'day': 5, 'sunny': 6, 'ruiny': 7}
[[2, 3, 4, 1, 5], [1, 1, 1, 4, 1, 5, 1]]


# Dopelnienie

W celu uzykania tego samego rozmiaru i ksztaltu danych stosujemy operacje dopelnienia.

- prepadding (dopelnienie z przodu)


In [20]:
sentences = ['Today is a sunny day',
           'Today is a rainy day',
           'Is today a sunny day?',
           'I really like today is snow walk']

from tensorflow.keras.preprocessing.sequence import pad_sequences # konwertuje listy na te sama dlugosc 
tokenizer = Tokenizer(num_words=100, oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences)
print(padded)

[[ 0  0  2  3  4  6  5]
 [ 0  0  2  3  4  7  5]
 [ 0  0  3  2  4  6  5]
 [ 8  9 10  2  3 11 12]]


In [22]:
padded = pad_sequences(sequences, padding='post')
print(padded)

[[ 2  3  4  6  5  0  0]
 [ 2  3  4  7  5  0  0]
 [ 3  2  4  6  5  0  0]
 [ 8  9 10  2  3 11 12]]


In [23]:
padded = pad_sequences(sequences, padding='post', maxlen=6)
print(padded)

[[ 2  3  4  6  5  0]
 [ 2  3  4  7  5  0]
 [ 3  2  4  6  5  0]
 [ 9 10  2  3 11 12]]


In [24]:
padded = pad_sequences(sequences, padding='post', maxlen=6, truncating='post')
print(padded)

[[ 2  3  4  6  5  0]
 [ 2  3  4  7  5  0]
 [ 3  2  4  6  5  0]
 [ 8  9 10  2  3 11]]


# Slowa nieinformatywne, porzadkowanie tekstu
- slowa nieinformatywane (stopwords): the, and, but, znaczniki HTML, znaki interpunkcyjne, nazwy.

In [36]:
# usuniecie znacznikow 

sentence = '''<html style="font-family: Segoe UI;">
<head>
    <title>Dokument</title>
</head>
<body>
    <h1>Przykład</h1>
    <hr size="1" width="96%" align="center">
    Wikipedia: <a href=https://www.pl.wikipedia.org>kliknij. tutaj.!</a>
</body>
</html>
Ja mam, ty masz, my mamy, duzo pomyslow w glowie.'''

from bs4 import BeautifulSoup
soup = BeautifulSoup(sentence)
sentence = soup.get_text()
print(sentence)



Dokument


Przykład

    Wikipedia: kliknij. tutaj.!


Ja mam, ty masz, my mamy, duzo pomyslow w glowie.


In [38]:
# slowa nieinformatywae (stopwords)
stopwords = ['i', 'ja', 'ty', 'my', 'o', 'w', 'przy']

words = sentence.split()
filtered_sentence = ''
sentences = []
for word in words:
    if word not in stopwords:
        filtered_sentence = filtered_sentence + word + ' '
        
sentences.append(filtered_sentence)
print(sentences)

['Dokument Przykład Wikipedia: kliknij. tutaj.! Ja mam, masz, mamy, duzo pomyslow glowie. ']


In [39]:
# znaki interpunkcyjne 
import string 
table = str.maketrans('', '', string.punctuation)

words = sentence.split()
filtered_sentence = ''
sentences = []
for word in words:
    word = word.translate(table)
    if word not in stopwords:
        filtered_sentence = filtered_sentence + word + ' '
        
sentences.append(filtered_sentence)
print(sentences)

['Dokument Przykład Wikipedia kliknij tutaj Ja mam masz mamy duzo pomyslow glowie ']


In [40]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [50]:
table = str.maketrans('xyz', 'zyx', string.punctuation)
'xyzzzzzzzz....'.translate(table)

'zyxxxxxxxx'